In [ ]:
!pip install langchain langchain-openai openai duckduckgo-search youtube_search wikipedia langchainhub numexpr

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain_community.agent_toolkits import create_sql_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities import SQLDatabase

In [ ]:
# OPENAI KEY lesen
import os
try:
    from google.colab import userdata
    OPENAI_KEY = userdata.get('OPENAI_KEY')
except:
    OPENAI_KEY = os.getenv('OPENAI_KEY')
os.environ['OPENAI_API_KEY'] = OPENAI_KEY


In [ ]:
llm = ChatOpenAI(model = "gpt-4-1106-preview",temperature=0)

## SQL Datenbank-Agent

In [ ]:
!wget https://raw.githubusercontent.com/floleuerer/elvtr/main/session_08/Chinook_Sqlite.sqlite

In [ ]:
db = SQLDatabase.from_uri("sqlite:///Chinook_Sqlite.sqlite")
db.run("SELECT * FROM Artist LIMIT 10;")

In [ ]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

In [ ]:
agent_executor.invoke('Wie viele Alben hat Metallica verkauft?')

In [ ]:
agent_executor.invoke('Wie was ist das meistverkaufte Album von Metallica?')

## Agent mit Internetzugriff, Wikipedia und Taschenrechner

In [ ]:
# load some tools
tools = load_tools(['wikipedia', 'ddg-search','llm-math'], llm=llm)

# initialize agent with the tools, language model, and the type of agent
agent = initialize_agent(tools,
                         llm,
                         agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                         verbose=True
                         )

In [ ]:
print(agent.agent.llm_chain.prompt.template)

In [ ]:
agent.invoke('welches EM Spiel ist in Hamburg?', verbose=True)

In [ ]:
agent.invoke('Wie viele Alben hat Metallica verkauft?', verbose=True)

In [ ]:
agent.invoke('Wo spielt Manuel Neuer und wann ist das nächste Spiel der Mannschaft?', verbose=True)

## OpenAPI Spec

In [ ]:
from pathlib import Path
from langchain_community.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain_community.agent_toolkits.openapi import planner
import json
from langchain.requests import RequestsWrapper

In [ ]:
!wget https://raw.githubusercontent.com/floleuerer/elvtr/main/session_08/api.json

In [ ]:
openai_api_spec = reduce_openapi_spec(json.loads(Path('api.json').read_text()))

In [ ]:
spotify_agent = planner.create_openapi_agent(
    openai_api_spec,
    RequestsWrapper(),
    llm,
    allow_dangerous_requests=True
)
user_query = (
    "Your username is 'testuser123'. Ich muss einkaufen - packe das auf die Todoliste"
)
spotify_agent.invoke(user_query)